In [1]:
%pip install opendatasets

Note: you may need to restart the kernel to use updated packages.


In [148]:
import mlflow
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA, NMF
from random import randint
from sklearn.model_selection import RandomizedSearchCV

In [115]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("HousePrice")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1664632408127, experiment_id='3', last_update_time=1664632408127, lifecycle_stage='active', name='HousePrice', tags={}>

## Cargamos los datos

In [116]:
od.download(
    "https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data"
    )

Skipping, found downloaded files in ".\house-prices-advanced-regression-techniques" (use force=True to force download)


In [117]:
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")

In [118]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [119]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [120]:
X_train = train.loc[:, train.columns != "SalePrice"]
y_train = train["SalePrice"]

X_test = test.loc[:, test.columns != "SalePrice"]

## Creamos la etapa de preprocesamiento

In [125]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
    ("ss", StandardScaler()),
    ("pca", PCA())
    ]
)

cat_pipeline = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ("nmf", NMF())
    ]
)

column_transformer = ColumnTransformer([
    ("categorical", cat_pipeline, list(X_train.select_dtypes(exclude=np.number).columns)),
    ("numerical", num_pipeline, list(X_train.select_dtypes(np.number).columns))
    ]
)

## Instanciamos los modelos

### DecisionTree

In [145]:
modelo_tree = Pipeline([
    ("prepr", column_transformer),
    ("modelo", DecisionTreeRegressor())
    ]
)

tree_param_grid = {
    'prepr__numerical__pca__n_components': [15],
    'prepr__categorical__nmf__n_components':[15],
    'modelo__ccp_alpha': [0.0, 0.1, 0.3, 0.5],
    'modelo__criterion': ['squared_error', "friedman_mse"],
    'modelo__max_depth': [3,4,5,None],
    'modelo__max_features': ["sqrt", "log2", None],
}


### SupportVectorMachine

In [146]:
modelo_svm = Pipeline([
    ("prepr", column_transformer),
    ("modelo", SVR())
    ]
)

svm_param_grid = {
    'prepr__numerical__pca__n_components': [15],
    'prepr__categorical__nmf__n_components':[15],
    'modelo__C': [1.0, 0.5, 0.8, 0.2],
    'modelo__coef0': [0.0,  0.5],
    'modelo__epsilon': [0.1],
    'modelo__gamma': ['scale', 'auto'],
    'modelo__kernel': ['poly', 'rbf', 'sigmoid'],
}

### Random forest

In [147]:
modelo_forest = Pipeline([
    ("prepr", column_transformer),
    ("modelo", RandomForestRegressor())
    ]
)

forest_param_grid = {
    'prepr__numerical__pca__n_components': [15],
    'prepr__categorical__nmf__n_components':[15],
    'modelo__ccp_alpha': [0.0],
    'modelo__criterion': ["squared_error", "absolute_error", "poisson"],
    'modelo__max_features': ["sqrt", "log2", 1.0],
    'modelo__max_samples': [10, 20, 30, 40, None],
    'modelo__min_samples_split': [1, 2, 3],
    'modelo__n_estimators': [50,100,150,200],
    'modelo__n_jobs': [-1],
}

## Instanciamos las búsquedas de hiperparámetros

In [149]:
tree_optimization = RandomizedSearchCV(modelo_tree, tree_param_grid, n_jobs=-1, scoring="neg_mean_absolute_error", n_iter=100) 
forest_optimization = RandomizedSearchCV(modelo_svm, svm_param_grid, n_jobs=-1, scoring="neg_mean_absolute_error", n_iter=100) 
svm_optimization = RandomizedSearchCV(modelo_forest, forest_param_grid, n_jobs=-1, scoring="neg_mean_absolute_error", n_iter=100)

## Realizamos los experimentos

In [156]:
with mlflow.start_run():
    mlflow.log_param("model", "DecisionTree")
    mlflow.sklearn.autolog()
    tree_optimization.fit(X_train, y_train)
    mlflow.sklearn.autolog(disable=True)

2022/10/01 16:34:53 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `Pipeline(steps=[('prepr',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('nmf',
                                                                   NMF())]),
                                                  ['MSZoning', 'Street...`
c:\Users\Abelda-san\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 96 is smaller than n_iter=100. Running 96 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\Abelda-san\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1692: ConvergenceWarning: Maximum number of iterations 200 reached. Inc

In [ ]:
with mlflow.start_run():
    mlflow.log_param("model", "SupportVectorMachine")
    mlflow.sklearn.autolog()
    svm_optimization.fit(X_train, y_train)
    mlflow.sklearn.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    mlflow.log_param("model", "RandomForest")
    mlflow.sklearn.autolog()
    forest_optimization.fit(X_train, y_train)
    mlflow.sklearn.autolog(disable=True)

In [ ]:
# TODO: cambiar gridsearch por halvinggridsearch
# TODO: cambiar a MLFlow manual?